## COMPUTING PMI SCORES
in this notebook I will experiement with computing the PMI scores using duckDB

In [1]:
import duckdb

In [2]:
db_connection = duckdb.connect('example_db.duckdb')
db_connection.execute("SHOW TABLES").arrow()

pyarrow.Table
name: string
----
name: [["ngram_counter","ngram_counts_of_size_1","ngram_counts_of_size_2","ngram_counts_of_size_3","ngram_counts_of_size_4","ngram_counts_of_size_5"]]

In [3]:
ngram_counter_table_name = 'ngram_counter'

## add columns

In [4]:
# TODO: read about sql's views it might be more efficient.
# add_column_query = f'ALTER TABLE {ngram_counter_table_name} ADD COLUMN log_likelihood DOUBLE;'
# add_column_query = f'ALTER TABLE {ngram_counter_table_name} DROP COLUMN log_likelihood;'

#     ADD COLUMN min_segmentation_sum DOUBLE;
#     ADD CULUMN pmi_score DOUBLE;
# """
# # db_connection.execute(add_column_query)M

# # add_column_query = f'ALTER TABLE {ngram_counter_table_name} ADD COLUMN min_segmentation_sum DOUBLE;'
# add_column_query = f'ALTER TABLE {ngram_counter_table_name} DROP COLUMN min_segmentation_sum;'
# db_connection.execute(add_column_query)
# # add_column_query = f'ALTER TABLE {ngram_counter_table_name} ADD COLUMN pmi_score DOUBLE;'
# add_column_query = f'ALTER TABLE {ngram_counter_table_name} DROP COLUMN pmi_score;'
# db_connection.execute(add_column_query)

In [5]:
rel = db_connection.sql(f'SELECT * FROM {ngram_counter_table_name}')
rel

┌──────────────────────┬────────┐
│        ngram         │ count  │
│       varchar        │ int32  │
├──────────────────────┼────────┤
│ 2788                 │   1260 │
│ 1010                 │ 627689 │
│ 2002                 │ 188278 │
│ 2052                 │  35503 │
│ 2022                 │  50430 │
│ 13311                │    249 │
│ 2105                 │  18310 │
│ 1996                 │ 439426 │
│ 2542                 │   2311 │
│ 2282                 │  10507 │
│  ·                   │      · │
│  ·                   │      · │
│  ·                   │      · │
│ 2545,2788,3062,2046  │      1 │
│ 2788,3062,2046,2048  │      1 │
│ 3062,2046,2048,7236  │      1 │
│ 2046,2048,7236,1011  │      1 │
│ 2048,7236,1011,2216  │      1 │
│ 7236,1011,2216,2040  │      1 │
│ 1011,2216,2040,2020  │      1 │
│ 2216,2040,2020,2145  │      1 │
│ 2040,2020,2145,13879 │      1 │
│ 2020,2145,13879,2011 │      1 │
├──────────────────────┴────────┤
│ ? rows (>9999 rows, 20 shown) │
└─────────────

## compute the log liklihood into a new column to the database

In [6]:
max_count_query = rel.max('count')
max_count_query.show()

┌────────────┐
│ max(count) │
│   int32    │
├────────────┤
│     948506 │
└────────────┘



In [7]:
# TODO: DUMMY, replace with the real thing
dummy_count = 5_000_000
ngram_of_size_dict = {i: dummy_count for i in range(1, 6)}
ngram_of_size_dict

{1: 5000000, 2: 5000000, 3: 5000000, 4: 5000000, 5: 5000000}

In [8]:
# Now, I want to convert the table that I have into seperate table according to the different ngram-lengths.
# TODO: in the proper implementation, I want to do that from the start. there will not be a single large table.
# definitions
# CREATE TABLE t1(id INTEGER, j VARCHAR, PRIMARY KEY(id, j));
# UINTEGER
max_ngram_size = 5
table_name_format = 'ngram_counts_of_size_{ngram_size}'

In [9]:
def get_ngram_counts_of_size_table_name(ngram_size: int) -> str:
    return f'ngram_counts_of_size_{ngram_size}'

In [10]:
# create the new tables, for each ngram size. make sure that the ngram is represented as an array of unsigned ints
table_create_query_format = 'CREATE OR REPLACE TABLE {table_name}(' \
                            '{ngram_field_decleration}, count UINTEGER, PRIMARY KEY({ngram_field_key}));'
table_create_query_format

'CREATE OR REPLACE TABLE {table_name}({ngram_field_decleration}, count UINTEGER, PRIMARY KEY({ngram_field_key}));'

In [11]:
for ngram_size in range(1, max_ngram_size + 1):
    kwargs = {
        'table_name': table_name_format.format(ngram_size=ngram_size),
        'ngram_field_decleration': ', '.join(f'token_{i} UINTEGER' for i in range(1, ngram_size + 1)),
        'ngram_field_key': ', '.join(f'token_{i}' for i in range(1, ngram_size + 1)),
    }
    table_create_query = table_create_query_format.format(**kwargs)
    db_connection.execute(table_create_query)

In [31]:
db_connection.sql('SHOW TABLES;')

┌────────────────────────┐
│          name          │
│        varchar         │
├────────────────────────┤
│ ngram_counter          │
│ ngram_counts_of_size_1 │
│ ngram_counts_of_size_2 │
│ ngram_counts_of_size_3 │
│ ngram_counts_of_size_4 │
│ ngram_counts_of_size_5 │
└────────────────────────┘

In [13]:
# iterate through the main table in batches, and after each batch, insert to the appropriate table
main_table = 'ngram_counter'
main_table_data = db_connection.sql(f'SELECT * FROM {main_table}')
main_table_data

┌──────────────────────┬────────┐
│        ngram         │ count  │
│       varchar        │ int32  │
├──────────────────────┼────────┤
│ 2788                 │   1260 │
│ 1010                 │ 627689 │
│ 2002                 │ 188278 │
│ 2052                 │  35503 │
│ 2022                 │  50430 │
│ 13311                │    249 │
│ 2105                 │  18310 │
│ 1996                 │ 439426 │
│ 2542                 │   2311 │
│ 2282                 │  10507 │
│  ·                   │      · │
│  ·                   │      · │
│  ·                   │      · │
│ 2545,2788,3062,2046  │      1 │
│ 2788,3062,2046,2048  │      1 │
│ 3062,2046,2048,7236  │      1 │
│ 2046,2048,7236,1011  │      1 │
│ 2048,7236,1011,2216  │      1 │
│ 7236,1011,2216,2040  │      1 │
│ 1011,2216,2040,2020  │      1 │
│ 2216,2040,2020,2145  │      1 │
│ 2040,2020,2145,13879 │      1 │
│ 2020,2145,13879,2011 │      1 │
├──────────────────────┴────────┤
│ ? rows (>9999 rows, 20 shown) │
└─────────────

In [14]:
rows = [(1, 2, 3), (4, 5, 6)]
', '.join(map(str, rows))

'(1, 2, 3), (4, 5, 6)'

In [16]:
batch_size = 100_000
batch = main_table_data.fetchmany(batch_size)
batch
batch_i = 1
while len(batch) > 0 and batch_i < 10:
    print(f'processing batch {batch_i}...')
    split_by_ngram_size = [[] for i in range(1, max_ngram_size + 1)]
    
    # go over the batch
    for ngram_str, count in batch:
        # convert the keys into tuples of integers
        ngram = tuple(map(int, ngram_str.split(',')))
        # split those according to the index
        ngram_size = len(ngram)
        row = ngram + (count, )
        split_by_ngram_size[ngram_size - 1].append(row)
        
    # write to the appropriate table
    for ngram_size, rows in enumerate(split_by_ngram_size, 1):
        table_name = get_ngram_counts_of_size_table_name(ngram_size)
        insert_query = f'INSERT INTO {table_name} VALUES {", ".join(map(str, rows))};'
        db_connection.execute(insert_query)
        
    batch = main_table_data.fetchmany(batch_size)
    batch_i += 1

processing batch 1...
processing batch 2...
processing batch 3...
processing batch 4...
processing batch 5...
processing batch 6...
processing batch 7...
processing batch 8...
processing batch 9...


In [17]:
# make sure that the tables look like I expect them to look 
ngram_table_of_size_3 = get_ngram_counts_of_size_table_name(3)
db_connection.sql(f'SELECT * FROM {ngram_table_of_size_3}')

┌─────────┬─────────┬─────────┬────────┐
│ token_1 │ token_2 │ token_3 │ count  │
│ uint32  │ uint32  │ uint32  │ uint32 │
├─────────┼─────────┼─────────┼────────┤
│    2748 │    1010 │    2009 │    190 │
│    2009 │    2003 │    1012 │    626 │
│    1056 │    2903 │    2009 │    173 │
│    2903 │    2009 │    1012 │    154 │
│    2002 │    4711 │    2039 │     10 │
│    4711 │    2039 │    2010 │     17 │
│    2398 │    1999 │    9135 │      9 │
│    1999 │    9135 │    1012 │    131 │
│    2215 │    2000 │    2903 │     38 │
│    2000 │    2903 │    2505 │     12 │
│      ·  │      ·  │      ·  │      · │
│      ·  │      ·  │      ·  │      · │
│      ·  │      ·  │      ·  │      · │
│    2272 │    2000 │   10891 │      1 │
│    2000 │   10891 │    2411 │      1 │
│   10891 │    2411 │    1029 │      1 │
│    2411 │    1029 │    1005 │     15 │
│    1036 │    1037 │    3232 │     26 │
│    3232 │    1997 │    2335 │    111 │
│    1997 │    2335 │    1037 │      7 │
│    2335 │    1

In [18]:
ngram_table_of_size_4 = get_ngram_counts_of_size_table_name(4)
db_connection.sql(f'SELECT * FROM {ngram_table_of_size_4}')

┌─────────┬─────────┬─────────┬─────────┬────────┐
│ token_1 │ token_2 │ token_3 │ token_4 │ count  │
│ uint32  │ uint32  │ uint32  │ uint32  │ uint32 │
├─────────┼─────────┼─────────┼─────────┼────────┤
│    1036 │    2748 │    1010 │    2009 │    108 │
│    2748 │    1010 │    2009 │    2003 │     64 │
│    1010 │    2009 │    2003 │    1012 │     77 │
│    2009 │    2003 │    1012 │    1005 │    218 │
│    1005 │    1056 │    2903 │    2009 │    173 │
│    1056 │    2903 │    2009 │    1012 │     79 │
│    2903 │    2009 │    1012 │    1005 │     25 │
│    2002 │    4711 │    2039 │    2010 │      9 │
│    4711 │    2039 │    2010 │    2398 │     13 │
│    2010 │    2398 │    1999 │    9135 │      5 │
│      ·  │      ·  │      ·  │      ·  │      · │
│      ·  │      ·  │      ·  │      ·  │      · │
│      ·  │      ·  │      ·  │      ·  │      · │
│    2032 │    1999 │    1996 │    2690 │      8 │
│    1996 │    2690 │    1997 │    8057 │      1 │
│    2690 │    1997 │    8057 │

In [19]:
import math

In [21]:
# compute log likelihood
# TODO: maybe I can add column and set the values with a single command?
ngram_table_of_len_1 = get_ngram_counts_of_size_table_name(1)
log_total_count = math.log(ngram_of_size_dict[1])
compute_log_likelihood_query = f'SELECT token_1, ln(count) - {log_total_count} AS log_likelihood FROM {ngram_table_of_len_1}'
log_likelihood_rel_1 = db_connection.sql(compute_log_likelihood_query)
log_likelihood_rel_1

┌─────────┬─────────────────────┐
│ token_1 │   log_likelihood    │
│ uint32  │       double        │
├─────────┼─────────────────────┤
│    3271 │   -8.49739056412006 │
│    6594 │  -9.915560133770398 │
│    8881 │  -9.091668842258686 │
│   11633 │ -11.094215130112046 │
│   10849 │  -9.446062705497255 │
│   16095 │ -11.068239643708786 │
│   10280 │ -11.364505459851959 │
│    5810 │   -9.51415182635785 │
│    6024 │  -9.860428063075684 │
│   13523 │ -10.356044268178145 │
│     ·   │           ·         │
│     ·   │           ·         │
│     ·   │           ·         │
│   17951 │  -9.991226466844136 │
│   23078 │ -11.030499315725937 │
│   29115 │  -13.34550692871854 │
│    4973 │ -12.246894640050431 │
│   19886 │ -13.479038321343063 │
│    7718 │ -11.068239643708786 │
│   28607 │  -11.53312817228775 │
│   28344 │ -13.633189001170322 │
│   19319 │ -11.928440908931897 │
│    9013 │  -10.65426384593271 │
├─────────┴─────────────────────┤
│     7484 rows (20 shown)      │
└─────────────

In [22]:
# TODO: maybe at the start of the table I want to add those columns? this will remove the need to add it afterwords.
# TODO: maybe we will want to set the count to uint64
def add_log_likelihood_to_table(ngram_size: int):
    table_name = get_ngram_counts_of_size_table_name(ngram_size)
    
    add_col_to_table_query = f"ALTER TABLE {table_name} ADD COLUMN log_likelihood DOUBLE;"
    db_connection.execute(add_col_to_table_query)
    
    log_total_count = math.log(ngram_of_size_dict[ngram_size])
    update_table_query = f"UPDATE {table_name} SET log_likelihood = ln(count) - {log_total_count};"
    db_connection.execute(update_table_query)

In [23]:
add_log_likelihood_to_table(1)
db_connection.sql(f'SELECT * FROM {get_ngram_counts_of_size_table_name(1)}')

┌─────────┬────────┬─────────────────────┐
│ token_1 │ count  │   log_likelihood    │
│ uint32  │ uint32 │       double        │
├─────────┼────────┼─────────────────────┤
│    3271 │   1020 │   -8.49739056412006 │
│    6594 │    247 │  -9.915560133770398 │
│    8881 │    563 │  -9.091668842258686 │
│   11633 │     76 │ -11.094215130112046 │
│   10849 │    395 │  -9.446062705497255 │
│   16095 │     78 │ -11.068239643708786 │
│   10280 │     58 │ -11.364505459851959 │
│    5810 │    369 │   -9.51415182635785 │
│    6024 │    261 │  -9.860428063075684 │
│   13523 │    159 │ -10.356044268178145 │
│     ·   │     ·  │           ·         │
│     ·   │     ·  │           ·         │
│     ·   │     ·  │           ·         │
│   17951 │    229 │  -9.991226466844136 │
│   23078 │     81 │ -11.030499315725937 │
│   29115 │      8 │  -13.34550692871854 │
│    4973 │     24 │ -12.246894640050431 │
│   19886 │      7 │ -13.479038321343063 │
│    7718 │     78 │ -11.068239643708786 │
│   28607 │

In [25]:
for i in range(2, max_ngram_size + 1):
    add_log_likelihood_to_table(i)

In [26]:
db_connection.sql(f'SELECT * FROM {get_ngram_counts_of_size_table_name(3)}')

┌─────────┬─────────┬─────────┬────────┬─────────────────────┐
│ token_1 │ token_2 │ token_3 │ count  │   log_likelihood    │
│ uint32  │ uint32  │ uint32  │ uint32 │       double        │
├─────────┼─────────┼─────────┼────────┼─────────────────────┤
│    2748 │    1010 │    2009 │    190 │ -10.177924398237892 │
│    2009 │    2003 │    1012 │    626 │   -8.98559809929828 │
│    1056 │    2903 │    2009 │    173 │ -10.271656875900597 │
│    2903 │    2009 │    1012 │    154 │ -10.387995867984747 │
│    2002 │    4711 │    2039 │     10 │  -13.12236337740433 │
│    4711 │    2039 │    2010 │     17 │ -12.591735126342162 │
│    2398 │    1999 │    9135 │      9 │ -13.227723893062157 │
│    1999 │    9135 │    1012 │    131 │ -10.549751147197226 │
│    2215 │    2000 │    2903 │     38 │ -11.787362310671991 │
│    2000 │    2903 │    2505 │     12 │ -12.940041820610377 │
│      ·  │      ·  │      ·  │      · │          ·          │
│      ·  │      ·  │      ·  │      · │          ·    

In [27]:
# compute the dynamic programming term into a new column
# I want to do that with batches, maybe using pyarrow tables or numpy, and then update.
# lets write the computation with SQL explicitly for ngram_size=1,2,3. then generalize it.

In [ ]:
# Pardon the terible name :( 
new_col_name = 'max_sum_segmentations_log_likelihood'

In [28]:
# ngram_size = 1
table_name = get_ngram_counts_of_size_table_name(1)
add_col_to_table_query = f"ALTER TABLE {table_name} ADD COLUMN {new_col_name} DOUBLE;"
db_connection.execute(add_col_to_table_query)

update_table_query = f"UPDATE {table_name} SET {new_col_name} = log_likelihood;"
db_connection.execute(update_table_query)

db_connection.sql(f"SELECT * FROM {table_name}")

┌─────────┬────────┬─────────────────────┬──────────────────────────────────────┐
│ token_1 │ count  │   log_likelihood    │ max_sum_segmentations_log_likelihood │
│ uint32  │ uint32 │       double        │                double                │
├─────────┼────────┼─────────────────────┼──────────────────────────────────────┤
│    3271 │   1020 │   -8.49739056412006 │                    -8.49739056412006 │
│    6594 │    247 │  -9.915560133770398 │                   -9.915560133770398 │
│    8881 │    563 │  -9.091668842258686 │                   -9.091668842258686 │
│   11633 │     76 │ -11.094215130112046 │                  -11.094215130112046 │
│   10849 │    395 │  -9.446062705497255 │                   -9.446062705497255 │
│   16095 │     78 │ -11.068239643708786 │                  -11.068239643708786 │
│   10280 │     58 │ -11.364505459851959 │                  -11.364505459851959 │
│    5810 │    369 │   -9.51415182635785 │                    -9.51415182635785 │
│    6024 │    2

In [29]:
# ngram_size = 2
# TODO: maybe give this some default value so I can take the maximal value every time, without having to create a specieal
#  initail stage
table_name = get_ngram_counts_of_size_table_name(2)
add_col_to_table_query = f"ALTER TABLE {table_name} ADD COLUMN {new_col_name} DOUBLE;"
db_connection.execute(add_col_to_table_query)

In [46]:
# select the new_col_name of the first and second token that match the original table
# TODO: something with my joins is not right. I need to fix this. I don't have the same number of columns. 
table_1 = get_ngram_counts_of_size_table_name(1)
table_2 = get_ngram_counts_of_size_table_name(2)
query1 = f"""
SELECT {table_1}.{new_col_name} AS {new_col_name}_1
    FROM {table_2} JOIN {table_1} ON ({table_2}.token_1 = {table_1}.token_1)
"""
query2 = f"""
SELECT {table_1}.{new_col_name} AS {new_col_name}_2
    FROM {table_2} JOIN {table_1} ON ({table_2}.token_2 = {table_1}.token_1)
"""
query3 = f"""
SELECT {new_col_name}_1 + {new_col_name}_2 AS {new_col_name}
    FROM ({query1}) POSITIONAL JOIN ({query2})
"""
rel = db_connection.sql(query3)
rel

┌──────────────────────────────────────┐
│ max_sum_segmentations_log_likelihood │
│                double                │
├──────────────────────────────────────┤
│                   -16.99478112824012 │
│                  -19.831120267540797 │
│                  -18.183337684517372 │
│                   -22.18843026022409 │
│                   -18.89212541099451 │
│                  -22.136479287417572 │
│                  -22.729010919703917 │
│                    -19.0283036527157 │
│                  -19.720856126151368 │
│                   -20.71208853635629 │
│                            ·         │
│                            ·         │
│                            ·         │
│                  -18.070812915946615 │
│                   -21.43775923552596 │
│                   -18.72857640813465 │
│                   -19.17152483625742 │
│                  -19.819015298167063 │
│                  -19.558216829857656 │
│                   -21.42816176454091 │
│               

In [47]:
len(rel)

24709

In [48]:
rel = db_connection.sql(f"SELECT * FROM {table_2}")
rel

┌─────────┬─────────┬────────┬─────────────────────┬──────────────────────────────────────┐
│ token_1 │ token_2 │ count  │   log_likelihood    │ max_sum_segmentations_log_likelihood │
│ uint32  │ uint32  │ uint32 │       double        │                double                │
├─────────┼─────────┼────────┼─────────────────────┼──────────────────────────────────────┤
│    2903 │    2009 │    408 │  -9.413681295994216 │                                 NULL │
│    1999 │    9135 │    188 │ -10.188506507568427 │                                 NULL │
│    2903 │    2505 │     31 │ -11.990961265913231 │                                 NULL │
│    2505 │    8764 │      1 │ -15.424948470398377 │                                 NULL │
│    8764 │    2056 │    102 │ -10.799975657114107 │                                 NULL │
│    2926 │    2008 │      8 │  -13.34550692871854 │                                 NULL │
│    2001 │    2017 │    117 │  -10.66277453560062 │                            

In [49]:
len(rel)

96333

In [45]:
# now, update the table from those values
# TODO: maybe consider not saving intermidiate results by using views. will it make things more efficient?
#    in the docs, it is mentioned that a view will be re-computed every time that it is mentioned. 
# TODO: I will probably want to set a default value so that 
update_table_2_query = f"""
UPDATE {table_2} 
SET {new_col_name} = x.{new_col_name}
FROM ({query3}) AS x
"""
db_connection.execute(update_table_2_query)
db_connection.sql(f"SELECT * FROM {table_2}")

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

┌─────────┬─────────┬────────┬─────────────────────┬──────────────────────────────────────┐
│ token_1 │ token_2 │ count  │   log_likelihood    │ max_sum_segmentations_log_likelihood │
│ uint32  │ uint32  │ uint32 │       double        │                double                │
├─────────┼─────────┼────────┼─────────────────────┼──────────────────────────────────────┤
│    2903 │    2009 │    408 │  -9.413681295994216 │                                 NULL │
│    1999 │    9135 │    188 │ -10.188506507568427 │                                 NULL │
│    2903 │    2505 │     31 │ -11.990961265913231 │                                 NULL │
│    2505 │    8764 │      1 │ -15.424948470398377 │                                 NULL │
│    8764 │    2056 │    102 │ -10.799975657114107 │                                 NULL │
│    2926 │    2008 │      8 │  -13.34550692871854 │                                 NULL │
│    2001 │    2017 │    117 │  -10.66277453560062 │                            

In [ ]:
update_table_query = f"UPDATE {table_name} SET {new_col_name} = log_likelihood;"
db_connection.execute(update_table_query)
db_connection.sql(f"SELECT * FROM {table_name}")

In [66]:
# play with some example data. I want to simulate the case for 2-ngrams pmi score computation
import pyarrow as pa
table1 = {
    'token_1': [1, 2, 3],
    'x':   [1, 2, 3]
}
table1 = pa.Table.from_pydict(table1)
db_connection.sql("SELECT * FROM table1")

┌─────────┬───────┐
│ token_1 │   x   │
│  int64  │ int64 │
├─────────┼───────┤
│       1 │     1 │
│       2 │     2 │
│       3 │     3 │
└─────────┴───────┘

In [67]:
table2 = {
    'token_1': [1, 1, 1, 2, 3, 3],
    'token_2': [1, 2, 3, 1, 2, 1],
    'x':       [0, 0, 0, 0, 0, 0]
}
table2 = pa.Table.from_pydict(table2)
db_connection.sql("SELECT * FROM table2")

┌─────────┬─────────┬───────┐
│ token_1 │ token_2 │   x   │
│  int64  │  int64  │ int64 │
├─────────┼─────────┼───────┤
│       1 │       1 │     0 │
│       1 │       2 │     0 │
│       1 │       3 │     0 │
│       2 │       1 │     0 │
│       3 │       2 │     0 │
│       3 │       1 │     0 │
└─────────┴─────────┴───────┘

In [72]:
# I want to create a new table with an sql query, where there will be the sum of the counts from the first and second token.
query1 = f"""
SELECT table1.x AS a
FROM table2 JOIN table1 ON table2.token_1 = table1.token_1
"""
query2 = f"""
SELECT table1.x AS b
FROM table2 JOIN table1 ON table2.token_2 = table1.token_1
"""
query3 = f"""
SELECT a + b as sumsum FROM ({query1}) POSITIONAL JOIN ({query2})
"""
db_connection.sql(query3)

┌────────┐
│ sumsum │
│ int64  │
├────────┤
│      2 │
│      3 │
│      4 │
│      3 │
│      5 │
│      4 │
└────────┘

In [ ]:
# TODO: table3 = ...
# ngram_of_size_count = {
#     1: pa.compute.sum(table1['count']),
#     2: pa.compute.sum(table2['count'])
# }

In [ ]:
# compute pmi scores into a new column